# Projeto 1 - Modelo de Classificação

### Projeto realizado por :

#### 
* Cosmin Trandafir - 57101
* Martim Baptista - 56323
* João Serafim - 56376
* Martim Paraíba - 56273
***

#### Counter de horas I guess :{}

* Cosmin Trandafir - 4h
* Martim Baptista - 
* João Serafim - 3h
* Martim Paraíba - 
***

Questoes: 
- Devemos apagar colunas que possuem demasiados valores a zero?
- Devemos normalizar primeiro ou inputar primeiro?
- Devemos testar e mostrar os testes de todos os hyperparametros ou apenas os principais?

- Devemos testar com IterativeImputer?

### Neste projeto vamos usar o dataset: ***biodegradable_a.cvs*** 

In [4]:
import pandas as pd

#Load biodegradable dataset

bio_df = pd.read_csv("biodegradable_a.csv")
bio_df

,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,NaN,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,NaN,2.0,...,0.0,0.0,1.0,NaN,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,NaN,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,NaN,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,NaN,2.0,3.0,...,0.0,0.0,0.0,NaN,3.059198,0.0,7.908802,0.0,0.0,RB


## Generic functions and Imports

In [5]:
from sklearn import impute
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [6]:
def runner(scaler, imputer, modeler, X_train, y_train, X_test):
    #this is the heavy duty modeler, outputing predictions
    imputer.fit(X_train)
    Xt_train = imputer.transform(X_train)
    Xt_test  = imputer.transform(X_test)
    scaler.fit(Xt_train)
    Xt_train = scaler.transform(Xt_train)
    Xt_test  = scaler.transform(Xt_test)
    modeler.fit(Xt_train, y_train)
    return modeler.predict(Xt_test)

In [7]:
def final_scores(y_test, y_pred):
    # Evaluate the performance of the model using various metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)

    print("Accuracy: {:.4f}".format(accuracy))
    print("Precision:{:.4f}".format(precision))
    print("Recall:{:.4f}".format(recall))
    print("F1 score:{:.4f}".format(f1))
    print("Confusion matrix:\n", confusion_mat)


## Inicialization 

In [8]:
# Divide Freatures and Class columns for preprocessing
X = bio_df.iloc[:, :-1]
y = bio_df.iloc[:, -1]

# Encode string classes to a numeric value for Imputer
le = preprocessing.LabelEncoder()
df_encoded_classes = le.fit_transform(y)

## Data Imputation


***
### Simple Imputer

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
print("Simple Imputer")
#mean
# Replace NaN values with mean value of k-nearest neighbors using KNNImputer
simple_imputer = SimpleImputer(strategy = "mean")
df_imputed_simple = simple_imputer.fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_imputed_simple, df_encoded_classes, test_size=0.33)

# Create a logistic regression model
clf = LogisticRegression(max_iter=10000)

# Train the model on the training set
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the model using various metrics
final_scores(y_test, y_pred)
    
scores = cross_val_score(clf, df_imputed_simple, df_encoded_classes, cv=10)
print("Cross-validation scores:", scores)
print("Average score:{:.4f}".format(scores.mean()))
print("----------------------------------------")
print("Now using the median for the imputation")
print("----------------------------------------")

#median
# Replace NaN values with mean value of k-nearest neighbors using KNNImputer
simple_imputer = SimpleImputer(strategy = "median")
df_imputed_simple = simple_imputer.fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_imputed_simple, df_encoded_classes, test_size=0.33)

# Create a logistic regression model
clf = LogisticRegression(max_iter=10000)

# Train the model on the training set
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the model using various metrics
final_scores(y_test, y_pred)
scores = cross_val_score(clf, df_imputed_simple, df_encoded_classes, cv=10)
print("Cross-validation scores:", scores)
print("Average score:{:.4f}".format(scores.mean()))

Simple Imputer
Accuracy: 0.9456
Precision:0.9510
Recall:0.9857
F1 score:0.9680
Confusion matrix:
 [[ 184   64]
 [  18 1241]]
Cross-validation scores: [0.87527352 0.95185996 0.96498906 0.96936543 0.96491228 0.96491228
 0.93421053 0.95175439 0.96491228 0.94517544]
Average score:0.9487
----------------------------------------
Now using the median for the imputation
----------------------------------------
Accuracy: 0.9463
Precision:0.9478
Recall:0.9905
F1 score:0.9687
Confusion matrix:
 [[ 173   69]
 [  12 1253]]
Cross-validation scores: [0.87964989 0.94967177 0.96498906 0.96280088 0.9627193  0.96710526
 0.92763158 0.95175439 0.9627193  0.94517544]
Average score:0.9474


***
### KNN Imputer

In [21]:
from sklearn.impute import KNNImputer
print("KNN Imputer")
# Replace NaN values with mean value of k-nearest neighbors using KNNImputer
knn_imputer = KNNImputer()
df_imputed_knn = knn_imputer.fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_imputed_knn, df_encoded_classes, test_size=0.33)

# Create a logistic regression model
clf = LogisticRegression(max_iter=10000)

# Train the model on the training set
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the model using various metrics
final_scores(y_test, y_pred)

scores = cross_val_score(clf, df_imputed_knn, df_encoded_classes, cv=10)
print("Cross-validation scores:", scores)
print("Average score:{:.4f}".format(scores.mean()))

KNN Imputer
Accuracy: 0.9562
Precision:0.9630
Recall:0.9858
F1 score:0.9743
Confusion matrix:
 [[ 190   48]
 [  18 1251]]
Cross-validation scores: [0.86870897 0.94967177 0.96280088 0.97155361 0.96491228 0.96929825
 0.9254386  0.95833333 0.96491228 0.94736842]
Average score:0.9483


***
### Iterative Imputer

In [22]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

print("Iterative Imputer")
# Replace NaN values with mean value of k-nearest neighbors using KNNImputer
iter_imputer = IterativeImputer(random_state = 0)
df_imputed_iter = iter_imputer.fit_transform(X)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_imputed_iter, df_encoded_classes, test_size=0.33)

# Create a logistic regression model
clf = LogisticRegression(max_iter=10000)

# Train the model on the training set
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the model using various metrics
final_scores(y_test, y_pred)

scores = cross_val_score(clf, df_imputed_iter, df_encoded_classes, cv=10)
print("Cross-validation scores:", scores)
print("Average score:{:.4f}".format(scores.mean()))

Iterative Imputer
Accuracy: 0.9489
Precision:0.9530
Recall:0.9872
F1 score:0.9698
Confusion matrix:
 [[ 192   61]
 [  16 1238]]
Cross-validation scores: [0.87089716 0.94967177 0.96061269 0.97155361 0.9627193  0.96710526
 0.92105263 0.95833333 0.96491228 0.94517544]
Average score:0.9472


## Data Scaling

In [12]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, PowerTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Using a simple imputer TODO use the best
simple_imputer = SimpleImputer(strategy = "median")
df_imputed_simple = simple_imputer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(df_imputed_simple, y, test_size=0.33)
X_train.shape, X_test.shape

scalers = [(MinMaxScaler(),"MinMaxScaler"), (StandardScaler(),"StandardScaler"), (Normalizer(),"Normalizer"), (PowerTransformer(),"PowerTransformer")]

for scaler,name in scalers:
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = DecisionTreeClassifier()
    clf.fit(X_train_scaled, y_train)

    y_pred = clf.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='RB')
    recall = recall_score(y_test, y_pred, pos_label='RB')
    cm = confusion_matrix(y_test, y_pred)
    print("Results for:", name)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{cm}\n")

Results for: MinMaxScaler
Accuracy: 0.9589
Precision: 0.9711
Recall: 0.9803
Confusion Matrix:
[[ 203   37]
 [  25 1242]]

Results for: StandardScaler
Accuracy: 0.9529
Precision: 0.9709
Recall: 0.9732
Confusion Matrix:
[[ 203   37]
 [  34 1233]]

Results for: Normalizer
Accuracy: 0.9542
Precision: 0.9731
Recall: 0.9724
Confusion Matrix:
[[ 206   34]
 [  35 1232]]

Results for: PowerTransformer
Accuracy: 0.9536
Precision: 0.9679
Recall: 0.9771
Confusion Matrix:
[[ 199   41]
 [  29 1238]]



## Feature Selection

In [57]:
import numpy as np
import pandas as pd
from sklearn.metrics import explained_variance_score, mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold
def model_testing(X_train, X_test, y_train, y_test, feature_selection=None):
    # select top 5 most correlated variables with y, if feature_selection is not None
    if feature_selection == 'correlation':
        corr_matrix = np.corrcoef(np.hstack((y_train.reshape((len(y_train), 1)), X_train)).T)
        corr_with_y = corr_matrix[0, 1:]
        correlationli = list(corr_with_y)
        sorted_indices = sorted(range(len(correlationli)), key=lambda k: correlationli[k], reverse=True)
        top5 = sorted_indices[:5]
        print("Top 5 by correlation:", top5)
        X_train = X_train[:, top5]
        X_test = X_test[:, top5]

    # train models
    dtr = DecisionTreeRegressor(max_depth=5)
    dtr.fit(X_train, y_train)

    lmr = LinearRegression()
    lmr.fit(X_train, y_train)

    # evaluate models
    dt_preds = dtr.predict(X_test)
    lr_preds = lmr.predict(X_test)

    print("RVE DTs: %7.4f" % explained_variance_score(y_test, dt_preds))
    print("RVE LRs: %7.4f" % explained_variance_score(y_test, lr_preds))

### Analysis of correlation

In [58]:
X_train, X_test, y_train, y_test = train_test_split(df_imputed_simple, df_encoded_classes, test_size=0.33)
    
# use all variables
print("------Using all variables------")
model_testing(X_train, X_test, y_train, y_test)
print("------Using the top5------")
# use top 5 most correlated variables
model_testing(X_train, X_test, y_train, y_test, feature_selection='correlation')


------Using all variables------
RVE DTs:  0.5790
RVE LRs:  0.5238
------Using the top5------
Top 5 by correlation: [13, 17, 9, 30, 36]
RVE DTs:  0.1669
RVE LRs:  0.0758


### Stepwise - Forward and Backward

In [59]:
from sklearn.feature_selection import SequentialFeatureSelector

N,M=X_train.shape
#LR
lmr=LinearRegression()
sfs = SequentialFeatureSelector(lmr, n_features_to_select=5)
sfs.fit(X_train, y_train)

#get the relevant columns
features=sfs.get_support()
Features_selected =np.arange(M)[features]
print("The features selected are columns: ", Features_selected)

nX_train=sfs.transform(X_train)
nX_test=sfs.transform(X_test)

model_testing(nX_train, nX_test, y_train, y_test)
#--------------------------------------------------------------------
#DT Forward
dtr = DecisionTreeRegressor(max_depth=3)
sfs = SequentialFeatureSelector(dtr, n_features_to_select=5, direction="forward")
sfs.fit(X_train, y_train)

#get the relevant columns
features=sfs.get_support()
Features_selected =np.arange(M)[features]
print("Decision tree: Forward")
print("The features selected are columns: ", Features_selected)

nX_train=sfs.transform(X_train)
nX_test=sfs.transform(X_test)

model_testing(nX_train, nX_test, y_train, y_test)
#-------------------------------------------------------------------- takes a bit longer
#DT backward
dtr = DecisionTreeRegressor(max_depth=3)
sfs = SequentialFeatureSelector(dtr, n_features_to_select=5, direction="backward")
sfs.fit(X_train, y_train)

#get the relevant columns
features=sfs.get_support()
Features_selected =np.arange(M)[features]
print("Decision tree: Backward")
print("The features selected are columns: ", Features_selected)

nX_train=sfs.transform(X_train)
nX_test=sfs.transform(X_test)

model_testing(nX_train, nX_test, y_train, y_test)

The features selected are columns:  [ 2  5 19 21 33]
RVE DTs:  0.5559
RVE LRs:  0.4579
Decision tree: Forward
The features selected are columns:  [ 2  5 11 29 33]
RVE DTs:  0.5902
RVE LRs:  0.4277
Decision tree: Backward
The features selected are columns:  [ 2  5 10 11 21]
RVE DTs:  0.5684
RVE LRs:  0.4298
